In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
import time,random,sys
from bs4 import BeautifulSoup
import datetime
import locale,requests
locale.setlocale(locale.LC_ALL, '')


'Turkish_Turkey.1254'

In [2]:
def temizle(ilan):    
    ilan = ilan.replace("                \t","").replace("\n                        ","").replace("\n                    ","")
    ilan =ilan.replace("\n                ","").replace("\n            ","").replace("\xa0","").replace("\n","")
    return ilan
def sure(a):
    gun = int(a //(24*3600))
    zaman= a% (24*3600)
    saat = int(zaman//3600)
    zaman %= 3600
    dakika =int(zaman //60)
    zaman %=60
    saniye = int(zaman)
    return " {} gun {} saat {} dakika {} saniyede bitti.".format(gun,saat,dakika,saniye)
def bugun():
    an = datetime.datetime.now()
    gun =datetime.datetime.strftime(an, '%d')
    ay =datetime.datetime.strftime(an, '%B')
    tarih =gun + " " +ay
    return tarih
def tarih_kayit():
    with open("son_run.txt", "w",encoding="utf-8") as txt_file:    
        son_calistirma=bugun() 
        txt_file.write(son_calistirma)
def tarihal(al):
    gunler=[]
    f= open("son_run.txt", "r", encoding = "utf-8")
    oku =f.read() + " 2020"
    gun,ay,yil =oku.split()
    geri = al
    if not geri == al:
        for a in range(al+1):
            gunler.append(str(geri)+" "+ ay)
            geri -=1
            #time.sleep(1)
            if geri ==int(gun):
                break
    gunler.append(str(gun)+" "+ ay)
    return gunler
def harmanla():
    f=open("yeni.csv","r",encoding="utf-8")
    veriler= f.read().split("\n")
    hedef=set()
    for veri in veriler[:-1]:
        veri =veri.split("\t")
        kume= (veri[0],veri[1],veri[2],veri[3],veri[4],veri[5],veri[6],veri[7],veri[8])
        hedef.add(kume)
    #hedef.add(ekle)
    return hedef

def kayit(bilgiler):
    with open("yeni.csv", "w",encoding="utf-8") as txt_file:    
        for line in bilgiler:
            txt_file.write(str(line[0]) +"\t" +str(line[1]) +"\t" +str(line[2]) +"\t" +str(line[3]) +"\t" +str(line[4]) +"\t" +str(line[5]) +"\t" +str(line[6]) +"\t" +str(line[7])+ "\t" +str(line[8])+"\n")
    with open("yeni.txt", "w",encoding="utf-8") as txt_file:    
        for line in bilgiler:
            txt_file.write(str(line)+"\n")     

In [3]:
page = "https://www.sahibinden.com"
f = open("araba_markalari_altsayfalar.txt", "r")
satirlar = f.read().split('\n')
satirlar = random.sample(satirlar, len(satirlar))

In [4]:
def verileriGetir():
    tarih_kayit()
    deneme = 0 
    while True:
        print(bugun())
        say=0
        basla =time.time()    

        options = Options()  
        options.add_argument("--headless")
        driver = webdriver.Chrome(options=options)
        
        bilgiler =harmanla()
#         hata = set()
        deneme +=1
        for satir in satirlar:         
            for sayfa in range(0,251,50):            
#               yeniurl = page +satir +"?pagingOffset=" +str(sayfa)+ "&pagingSize=50&sorting=date_desc" 
                yeniurl = page +satir +"?viewType=Classic&pagingOffset=" +str(sayfa)+ "&pagingSize=50&sorting=date_desc" 
                #https://www.sahibinden.com/alfa-romeo-156?viewType=Classic&pagingOffset=50&pagingSize=50
                #https://www.sahibinden.com/alfa-romeo-156?viewType=Classic&pagingOffset=100&pagingSize=50
#                https://www.sahibinden.com/alfa-romeo-156
                say += 1
    
#                 if (say % 20)==0:
#                     time.sleep(5)
                    
                driver.get(yeniurl)
#                 rastgele = random.randint(1,6) 
#                 time.sleep(rastgele) 
                source = BeautifulSoup(driver.page_source,"lxml")               
                hepsi = source.select('tr.searchResultsItem:not(.nativeAd, .classicNativeAd)')            
                if (source.find("div",attrs ={"id":"qr-error-footer"})):
                    driver.delete_all_cookies()
#                     print(" Deneme: "+ str(deneme) + " ==>  " + str(say) + " ==>  " +yeniurl)
#                     hata.add(yeniurl)

                    for i in range(301):                
                        i +=1
                        sys.stdout.write("Olağan dışı erişim tespit ettik..." + str(301 -i)+ " saniye sonra tekrar deneyeceğim " +"\r")
                        #sys.stdout.flush()
                        time.sleep(1)                  

                else:
                    sys.stdout.write(" Deneme: "+ str(deneme) + " ==>  " + str(say) + " ==>  " +yeniurl + "\r")
                    #sys.stdout.flush()
                for araba in hepsi:
                    alan = araba.find_all("td",attrs ={"class":"searchResultsDateValue"})[0].find("span").text
                    #if alan == bugun():
                    myid= araba.find("div", attrs={"class":"action-wrapper"}).get("data-classified-id")
                    ust_baslik = source.find_all("a",attrs ={"class":"sprite breadcrumbItem"})[3].text
                    alt_baslik =  source.find_all("a",attrs ={"class":"sprite breadcrumbItem"})[4].text
                    if not araba.find_all("td",attrs ={"class":"searchResultsTagAttributeValue"}):
                        model = "null"
                    else:
                        model =araba.find_all("td",attrs ={"class":"searchResultsTagAttributeValue"})[0].text
                    yil = araba.find_all("td",attrs ={"class":"searchResultsAttributeValue"})[0].text
                    km = araba.find_all("td",attrs ={"class":"searchResultsAttributeValue"})[1].text
                    renk = araba.find_all("td",attrs ={"class":"searchResultsAttributeValue"})[2].text
                    fiyat = araba.find_all("td",attrs ={"class":"searchResultsPriceValue"})[0].text
                    ilanTarih =araba.find_all("td",attrs ={"class":"searchResultsDateValue"})[0].text
                    satısYeri =araba.find_all("td",attrs ={"class":"searchResultsLocationValue"})[0].text
                    myid =temizle(myid)
                    ust_baslik =temizle(ust_baslik)
                    alt_baslik=temizle(alt_baslik)
                    model =temizle(model)
                    yil = temizle(yil)
                    km = temizle(km)
                    renk = temizle(renk)
                    fiyat =temizle(fiyat)
                    ilanTarih = temizle(ilanTarih)
                    satısYeri =temizle(satısYeri)
                    #bilg = ust_baslik+"\t"  + alt_baslik+"\t"  + model+"\t"  + yil+"\t"  + km +"\t" + renk+"\t"  + fiyat +"\t"+ilanTarih +"\t" +satısYeri
                    bilgi = (myid,ust_baslik,alt_baslik,model, yil, km,renk,fiyat,ilanTarih,satısYeri)
                    bilgiler.add(bilgi)
                    #print(bilg)
            
                if not(source.find("ul",attrs ={"class":"pageNaviButtons"})):    
                    son_sayfa = source.find("ul",attrs ={"class":"pageNaviButtons"})    
                    break
            

        kalan = len(bilgiler)-len(harmanla())
        kayit(bilgiler)
        driver.delete_all_cookies()
        driver.quit()
        
        
        son = time.time()
        a = (son)-(basla)
        sure(a)
        print("Deneme:"+ str(deneme)+"==>  "+str(say)+" işlem"+sure(a)+ " Eklenen veri: "+ str(kalan) + " tane.                ")


In [ ]:
def attempts():
    while True:
        try:
            verileriGetir()
            break #breaks the loop when sucessful
        except ValueError:
            attempts() #recalls this function, starting back at the try statement
            break
attempts()

04 Temmuz
Deneme:1==>  1107 işlem 1 gun 2 saat 53 dakika 22 saniyede bitti. Eklenen veri: 28426 tane.                ngSize=50&sorting=date_descdescccescsc
05 Temmuz
Deneme:2==>  1277 işlem 0 gun 19 saat 33 dakika 49 saniyede bitti. Eklenen veri: 38252 tane.                gSize=50&sorting=date_descdescsccscsc
06 Temmuz


In [11]:
url1 = "https://www.sahibinden.com/sitemap/search/categories1.xml"
url2 = "https://www.sahibinden.com/sitemap/search/categories0.xml"
url3 = "https://www.sahibinden.com/sitemap/categories/categories1.xml"
url4 = "https://www.sahibinden.com/sitemap/categories/categories0.xml"
sifir_arac ="https://www.sahibinden.com/sitemap/oto360/sifir-araclar0.xml"
headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}
req = requests.get("https://www.sahibinden.com/sitemap/search/categories1.xml", headers=headers)
context = req.content
soup = BeautifulSoup(context, "html.parser")

In [13]:
hepsi = soup.find_all("url")

In [14]:
len(hepsi)

15000